In [3]:
import os
os.chdir('../')
%pwd

'/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars'

In [4]:
import tensorflow as tf

2023-09-26 13:16:37.078942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
tf.__version__

'2.12.0'

In [6]:
import box

In [7]:
from src.components.data_preparation import DataIngestion
from src.components.data_preprocesing import DataPreprocessing
from src.components.callbacks import PrepareCallback
from src.components.model import PrepareModel
from src.config.configuration import ConfigurationManager
from src.entity.config_entity import DataIngestionConfig, PrepareCallbackConfig, PrepareModelConfig


In [8]:
import glob

In [9]:
from src.entity.config_entity import DataPreprocessingConfig

In [11]:
try:
    config = ConfigurationManager()
    prepare_data_config = config.get_data_ingestion_config()

    prepare_dataset = DataIngestion(config=prepare_data_config)
    imgs_list, masks_list = prepare_dataset.prepare_data()

    preprocessing_config = config.get_data_processing_config()
    preprocessing = DataPreprocessing(preprocessing_config,imgs_list, masks_list)
    train, val = preprocessing.data_preprocessing()

except Exception as e:
    raise e



[2023-09-26 13:16:56,281: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-26 13:16:56,283: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-26 13:16:56,292: INFO: common: created directory at:artifacts]
[2023-09-26 13:16:56,293: INFO: common: created directory at:/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars/archive]


2023-09-26 13:16:56.348541: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-09-26 13:16:56.631972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5000]
	 [[{{node Placeholder/_0}}]]
2023-09-26 13:16:56.632315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [5000]
	 [[{{node Placeholder/_0}}]]


In [12]:
try:
    config = ConfigurationManager()
    prepare_model_config = config.get_prepare_model_config()
    prepare_model = PrepareModel(prepare_model_config)
    model = prepare_model.unet_model()
    model.summary()
except Exception as e:
    raise e

[2023-09-26 13:17:28,276: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-26 13:17:28,280: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-26 13:17:28,281: INFO: common: created directory at:artifacts]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 192, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 192, 256, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                          

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    epochs: int


In [11]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
from box import ConfigBox
from src import logger

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) :
                 
        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        self.config= ConfigBox(self.config[0]) # configbox is returning tuple so to avoid it we are again applying configbox to the dictionary at index 0
        self.params= ConfigBox(self.params[0])





    def get_prepare_training_config(self)-> ModelTrainingConfig:
        config = self.params



        prepare_training_config = PrepareModelConfig(
                            img_height = config.img_height,
                            img_width = config.img_width,
                            num_channels= config.num_channels,
                            epochs = config.EPOCHS)
        return prepare_training_config

    def get_prepare_callback_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks

        create_directories([config.root_dir])


        create_directories([ 
                            Path(config.checkpoint_model_filepath),
                            Path(config.tensorboard_root_log_dir)  ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
    
        )
        return prepare_callback_config

In [20]:
class Training:
    def __init__(self,config: ModelTrainingConfig):
        self.config = config
        

    def train(self, model, callbacklist, data):
        self.model = model((self.config.img_height,self.config.img_width, self.config.num_channels))



In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-09-26 13:17:39,572: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-26 13:17:39,574: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-26 13:17:39,575: INFO: common: created directory at:artifacts]
[2023-09-26 13:17:39,575: INFO: common: created directory at:artifacts/callbacks]
[2023-09-26 13:17:39,576: INFO: common: created directory at:artifacts/callbacks/checkpoints]
[2023-09-26 13:17:39,576: INFO: common: created directory at:artifacts/callbacks/tensorboard_log_dir]
artifacts/callbacks/tensorboard_log_dir/tb_logs_at_2023-09-26-13-17-39


In [12]:
import tensorflow as tf

In [14]:

model1 = model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
EPOCHS = 1
BUFFER_SIZE = 500
BATCH_SIZE = 64


In [2]:
tf.__version__

NameError: name 'tf' is not defined

In [15]:
model_history1 = model.fit(train, validation_data=val, epochs=1, callbacks=callback_list)

2023-09-26 13:17:52.591702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [5000]
	 [[{{node Placeholder/_1}}]]
2023-09-26 13:17:52.592094: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [5000]
	 [[{{node Placeholder/_1}}]]


: 